In [ ]:
import pandas as pd
import dask.dataframe as dd
from dask.distributed import Client
import fuzzywuzzy as fuzz


client = Client() #call the client from dask.distribute
client #initiate the client

client.scatter

name_1 = []
name_2 = []

In [ ]:
# Define matching function that will be used to provide a comparison of strings (drug names, strengths, and routes) for later merging of datasets
def match_name(name, list_names, min_score=0):
    # -1 score incase we don't get any matches
    max_score = -1
    # Returning empty name for no match as well
    max_name = ""
    # Iternating over all names in the other
    for name2 in list_names:
        print(name2)
        print(name)
        #Finding fuzzy match score
        score = fuzz.token_set_ratio(name, name2)
        # Checking if we are above our threshold and have a better score
        if (score > min_score) & (score > max_score):
            max_name = name2
            max_score = score
    return (max_name, max_score)

In [ ]:
# Runs the function above

# prices.set_index('ndc description')
# all_data.set_index('ndc_description_agg')
prices = prices.repartition(npartitions=prices.npartitions // 10)
all_data = all_data.repartition(npartitions=all_data.npartitions // 10)

# List for dicts for easy dataframe creation
dict_list = []
# iterating over our drugs to find a match
for name in prices['ndc description'].compute():
    # Use our method to find best match, we can set a threshold here
    print(name, all_data['ndc_description_agg'])
    
    match = match_name(name, all_data['ndc_description_agg'], 85)
    # New dict for storing data
    dict_ = {}
    dict_.update({'ndc description' : name})
    dict_.update({'ndc_description_agg' : match[0]})
    dict_.update({'score' : match[1]})
    dict_list.append(dict_)
    
merge_table1 = pd.DataFrame(dict_list)
# Display results
merge_table1